In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image, ImageDraw

from revolver.data.davis import DAVISInstSeg
from revolver.data.sparse import SparseSeg
from revolver.data.filter import SlugFilter
from revolver.data.conditional import ConditionalVideoSeg

In [ ]:
import os
import subprocess

root_dir = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip()
os.chdir(root_dir)

In [ ]:
def draw_circle(d, r, loc, color='white'):
    '''
    Draw circle of radius r at location loc
    on ImageDraw object d
    d = ImageDraw.Draw(im)
    '''
    y, x = loc[0], loc[1]
    d.ellipse((x-r, y-r, x+r, y+r), fill=tuple(color))
    
def load_and_show(ds, shot):
    plt.rcParams.update({'font.size': 16})
    fig, axes = plt.subplots(1, shot, figsize=(30, 10))
    qry, supp, tgt, _ = ds[0]
    for i, s in enumerate(supp):
        im, anno = np.copy(s[0]), s[1]
        im = Image.fromarray(im.astype(np.uint8))
        d = ImageDraw.Draw(im)
        for loc in zip(*np.where(anno != 0)):
            draw_circle(d, 10, loc[2:], color=ds.palette[loc[1]])
        axes[i].imshow(im)
        axes[i].set_title('Support')
    for _, ax in np.ndenumerate(axes):
        ax.set_axis_off()
        
    fig, axes = plt.subplots(1, 2, figsize=(30, 20))
    axes[0].imshow(qry)
    axes[0].set_title('Query')
    tgt = Image.fromarray(tgt.astype(np.uint8))
    tgt.putpalette(ds.palette)
    axes[1].imshow(tgt)
    axes[1].set_title('Target')
    for _, ax in np.ndenumerate(axes):
        ax.set_axis_off()


Visualize video segmentation tasks. During training, the support and query are sampled from anywhere in the sequence, and the instances that are annotated can vary. 

In [ ]:
# Configure support
count = 1 # number of sparse points per image
shot = 2 # number of images

In [ ]:
is_eval = False

sem_ds = DAVISInstSeg(split='val')
vid_dsets = {vid: SparseSeg(SlugFilter(sem_ds, 'vid', [vid]), count=count, static=is_eval) for vid in sem_ds.load_videos()}
cond_ds = ConditionalVideoSeg(sem_ds, vid_dsets, shot=shot, evaluation=is_eval)
load_and_show(cond_ds, shot)

During evaluation, support is fixed to be the first frame of the video, and all instances must be annotated. Query consists of the entire sequence.

In [ ]:
is_eval = True

sem_ds = DAVISInstSeg(split='val')
vid_dsets = {vid: SparseSeg(SlugFilter(sem_ds, 'vid', [vid]), count=count, static=is_eval) for vid in sem_ds.load_videos()}
cond_ds = ConditionalVideoSeg(sem_ds, vid_dsets, shot=shot, evaluation=is_eval)
load_and_show(cond_ds, shot)